In [ ]:
import sys,os
import ROOT as R
import glob

In [ ]:
R.TMVA.Tools.Instance()
R.gROOT.SetBatch()

In [ ]:
files = glob.glob("ntupleFiles/*.root")


In [ ]:
t = R.TChain("ntupledData")
for file in files: t.Add(file)

In [ ]:
outname="output.root"
out=R.TFile(outname,"RECREATE")

In [ ]:
analysis="Classification"
factory = ROOT.TMVA.Factory("factory",out,"!V:!Silent:Color:DrawProgressBar:Transformations=I;:AnalysisType=%s"%analysis)

Features Missing
"htCent"

In [ ]:
dataloader=R.TMVA.DataLoader("dataset")
features = ["h_pt","h_eta","h_phi","h_deta","h_dphi","mueta_1",
            "mueta_2","ncentJets","njets","mjj_1","mjj_2",
            "detajj_1","detajj_2","nbtagJets","metpt"]

In [ ]:
for x in features:
    dataloader.AddVariable(x)
dataloader.AddSpectator("h_mass")
dataloader.SetBackgroundWeightExpression("eweight")
dataloader.SetSignalWeightExpression("eweight")

In [ ]:
dataloader.AddSignalTree(t,1.0)
dataloader.AddBackgroundTree(t,1.0)
# signal mclabel is negative, background is positive
sigCut = R.TCut("(h_mass > 110 && h_mass < 150) && mclabel < 0")
bkgCut = R.TCut("(h_mass > 110 && h_mass < 150) && mclabel > 0")
dataloader. PrepareTrainingAndTestTree(sigCut,   bgCut, "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:!V");


In [ ]:
factory.BookMethod(dataloader, ROOT.TMVA.Types.kBDT, "BDTG",
                "!H:!V:NTrees=1200:MinNodeSize=3%:BoostType=Grad:Shrinkage=0.10:nCuts=40:MaxDepth=5:NodePurityLimit=0.99:SeparationType=SDivSqrtSPlusB:Pray"
                );

In [ ]:
factory.TrainAllMethods();
factory.TestAllMethods();
factory.EvaluateAllMethods();